In [8]:
import plotly.express as px
from sklearn.decomposition import PCA
import os
from src import DATA_DIR, PROJECT_ROOT, get_paper_info
from src.models.graph import Graph
from src.utils.io import load_json
import pandas as pd
import numpy as np

In [10]:
emb_path = os.path.join(DATA_DIR, 'emb', 'NeurIPS', '2024', '20250602_2037', 'emb.json')
emb = load_json(emb_path)

data_path = os.path.join(DATA_DIR, 'metadata', 'neurips_2023_2024.csv')
df = pd.read_csv(data_path)

clustering = os.path.join(PROJECT_ROOT, 'result', 'NeurIPS', '2024', '20250602_2158', 'clusters.csv')
cluster_df = pd.read_csv(clustering)

In [11]:
df = df[df['year'] == 2024]
df['emb'] = df['id'].map(emb)

In [12]:
# Create a mapping for each method
method_names = cluster_df['method'].tolist()
session_names = cluster_df.columns[1:]  # skip 'method'

# Build: method -> id -> cluster name
method_to_id_to_cluster = {}

for _, row in cluster_df.iterrows():
    method = row['method']
    id_to_cluster = {}
    for session in session_names:
        ids = str(row[session]).split(',') if pd.notna(row[session]) else []
        for id_ in ids:
            id_to_cluster[id_] = session  # assign paper to the session name
    method_to_id_to_cluster[method] = id_to_cluster

for method, id_to_cluster in method_to_id_to_cluster.items():
    df[method] = df['id'].map(id_to_cluster).fillna("None")  # or use np.nan

In [14]:
df.head(5)

,authors,publisher,title,url,year,abstract,session,pdf_url,openreview_url,id,forum_content,emb,Baseline,GreedyCohesive,KMedoids,KMeans
0,"['Tianyu He', 'Darshil Doshi', 'Aritra Das', '...",NeurIPS,Learning to grok_ Emergence of in-context lear...,https://neurips.cc/virtual/2024/oral/97968,2024,Large language models can solve tasks that we...,Oral Session 1A: Neuroscience and Intepretability,https://openreview.net/pdf?id=aVh9KRZdRk,https://openreview.net/forum?id=aVh9KRZdRk,aVh9KRZdRk,[{'content': {'title': {'value': 'Paper Decisi...,"[0.025920594792352154, -0.006532610617907675, ...",Oral Session 1A: Neuroscience and Intepretability,Oral Session 1A: Neuroscience and Intepretability,Oral Session 1A: Neuroscience and Intepretability,Oral Session 1A: Neuroscience and Intepretability
1,"['Jin Zhang', 'Ze Liu', 'Defu Lian', 'Enhong C...",NeurIPS,Generalization Error Bounds for Two-stage Reco...,https://neurips.cc/virtual/2024/oral/97958,2024,Two-stage recommender systems play a crucial ...,Oral Session 1D: Learning Theory,https://openreview.net/pdf?id=m1a4CrRJR7,https://openreview.net/forum?id=m1a4CrRJR7,m1a4CrRJR7,[{'content': {'title': {'value': 'Paper Decisi...,"[0.0005160157651468916, 0.009975187560102265, ...",Oral Session 1D: Learning Theory,Oral Session 1D: Learning Theory,Oral Session 1D: Learning Theory,Oral Session 1B: Human-AI Interaction
2,"['Aaron Defazio', 'Xingyu Yang', 'Ahmed Khaled...",NeurIPS,The Road Less Scheduled,https://neurips.cc/virtual/2024/oral/98003,2024,Existing learning rate schedules that do not ...,Oral Session 1C: Optimization and Learning Theory,https://openreview.net/pdf?id=0XeNkkENuI,https://openreview.net/forum?id=0XeNkkENuI,0XeNkkENuI,[{'content': {'title': {'value': 'Paper Decisi...,"[-0.0075764014618471265, -0.008110680956054817...",Oral Session 1C: Optimization and Learning Theory,Oral Session 6C: New Data,Oral Session 1C: Optimization and Learning Theory,Oral Session 1C: Optimization and Learning Theory
3,"['Rohan Alur', 'Manish Raghavan', 'Devavrat Sh...",NeurIPS,Human Expertise in Algorithmic Prediction,https://neurips.cc/virtual/2024/oral/97946,2024,We introduce a novel framework for incorporat...,Oral Session 1B: Human-AI Interaction,https://openreview.net/pdf?id=wpGJ2AX6SZ,https://openreview.net/forum?id=wpGJ2AX6SZ,wpGJ2AX6SZ,[{'content': {'comment': {'value': 'Thank you ...,"[0.006127413471889062, -0.0129554309300147, -0...",Oral Session 1B: Human-AI Interaction,Oral Session 1D: Learning Theory,Oral Session 1D: Learning Theory,Oral Session 1B: Human-AI Interaction
4,"['Shen Li', 'Yuyang Zhang', 'Zhaolin Ren', 'Cl...",NeurIPS,Enhancing Preference-based Linear Bandits via ...,https://neurips.cc/virtual/2024/oral/97969,2024,Interactive preference learning systems infer...,Oral Session 2A: Agents,https://openreview.net/pdf?id=aIPwlkdOut,https://openreview.net/forum?id=aIPwlkdOut,aIPwlkdOut,[{'content': {'title': {'value': 'Paper Decisi...,"[0.017408840711030996, -0.014277594156958388, ...",Oral Session 2A: Agents,Oral Session 1C: Optimization and Learning Theory,Oral Session 1D: Learning Theory,Oral Session 2A: Agents


In [13]:
X = np.vstack(df['emb'].values)
print(X.shape)

(72, 3072)


In [28]:
import numpy as np
from scipy.spatial import ConvexHull
import plotly.graph_objects as go

for method in method_names:
    # Create the base scatter plot
    fig = px.scatter(components, x=0, y=1,
                     title=f"{method} Clustering PCA",
                     hover_name=df['id'])

    # For each unique cluster (color)
    for cluster in df[method].unique():
        if cluster == "None":  # Skip if it's the "None" cluster
            continue

        # Get points belonging to this cluster
        mask = df[method] == cluster
        cluster_points = components[mask]

        if len(cluster_points) >= 3:  # Need at least 3 points for a convex hull
            # Compute the convex hull
            hull = ConvexHull(cluster_points)

            # Get the vertices of the hull in order
            vertices = cluster_points[hull.vertices]
            # Add first point at end to close the polygon
            vertices = np.vstack((vertices, vertices[0]))

            # Add the shape to the figure
            fig.add_trace(go.Scatter(
                x=vertices[:, 0],
                y=vertices[:, 1],
                mode='lines',
                line=dict(width=1),
                fill='toself',
                opacity=0.2,
                showlegend=False,
                hoverinfo='skip'
            ))

    fig.show()

In [35]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from scipy.spatial import ConvexHull
import plotly.express as px
import plotly.graph_objects as go


# Step 2: Run t-SNE
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
components = tsne.fit_transform(X)

# Step 3: Add to dataframe
df['tsne_0'] = components[:, 0]
df['tsne_1'] = components[:, 1]


In [36]:
# Step 4: Draw convex hulls for each method
for method in method_names:  # e.g. ["Baseline", "GreedyCohesive"]
    fig = px.scatter(
        df,
        x="tsne_0",
        y="tsne_1",
        color=method,
        hover_name="id",
        title=f"t-SNE Clustering — {method}"
    )

    for cluster in df[method].unique():
        if cluster == "None" or pd.isna(cluster):
            continue

        mask = df[method] == cluster
        cluster_points = df.loc[mask, ['tsne_0', 'tsne_1']].values

        if len(cluster_points) >= 3:
            hull = ConvexHull(cluster_points)
            vertices = cluster_points[hull.vertices]
            vertices = np.vstack([vertices, vertices[0]])  # close the loop

            fig.add_trace(go.Scatter(
                x=vertices[:, 0],
                y=vertices[:, 1],
                mode='lines',
                line=dict(width=1),
                fill='toself',
                fillcolor='rgba(0,0,0,0.1)',
                opacity=0.2,
                hoverinfo='skip',
                showlegend=False
            ))

    fig.show()
